### Question 1: [IPO] Withdrawn IPOs by Company Type

**What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**

From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn](https://stockanalysis.com/ipos/withdrawn/)), collect and process the data to find out which company type saw the most withdrawn IPO value.

#### Steps:
1. Use `pandas.read_html()` with the URL above to load the IPO withdrawal table into a DataFrame.
   *It is a similar process to Code Snippet 1 discussed at the livestream.*    You should get **99 entries**.
2. Create a new column called `Company Class`, categorizing company names based on patterns like:
   - “Acquisition Corp” or “Acquisition Corporation” → `Acq.Corp`
   - “Inc” or “Incorporated” → `Inc`
   - “Group” → `Group`
   - “Ltd” or “Limited” → `Limited`
   - “Holdings” → `Holdings`
   - Others → `Other`

  *  Order: Please follow the listed order of classes and assign the first matched value (e.g., for 'shenni holdings limited', you assign the 'Limited' class).

  * Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.

3. Define a new field `Avg. price` by parsing the `Price Range` field (create a function and apply it to the `Price Range` column). Examples:
   - '$8.00-$10.00' → `9.0`  
   - '$5.00' → `5.0`  
   - '-' → `None`
4. Convert `Shares Offered` to numeric, clean missing or invalid values.
5. Create a new column:  
   `Withdrawn Value = Shares Offered * Avg Price` (**71 non-null values**)
6. Group by `Company Class` and calculate total withdrawn value.
7. **Answer**: Which class had the highest **total** value of withdrawals?

In [ ]:
!pip install yfinance -q
# IMPORTS
import numpy as np
import pandas as pd
import requests


#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

def get_ipos() -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/withdrawn/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found ")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [ ]:
ipos = get_ipos()
ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [ ]:
def classify_company(name):
    if not isinstance(name, str):
        return 'Other'

    name = name.lower()
    words = set(name.replace(',', '').replace('.', '').split())

    if 'acquisition' in words and ('corp' in words or 'corporation' in words):
        return 'Acq.Corp'
    elif 'inc' in words or 'incorporated' in words:
        return 'Inc'
    elif 'group' in words:
        return 'Group'
    elif 'ltd' in words or 'limited' in words:
        return 'Limited'
    elif 'holdings' in words or 'holding' in words:
        return 'Holdings'
    else:
        return 'Other'

ipos['Company Class'] = ipos['Company Name'].apply(classify_company)






In [ ]:
def parse_price_range(price_range):
    if pd.isna(price_range) or price_range.strip() == '-' or price_range.strip() == '':
        return None
    price_range = price_range.replace('$', '')
    if '-' in price_range:
        try:
            low, high = map(float, price_range.split('-'))
            return (low + high) / 2
        except:
            return None
    try:
        return float(price_range)
    except:
        return None

ipos['Avg. price'] = ipos['Price Range'].apply(parse_price_range)



In [ ]:
ipos['Shares Offered'] = pd.to_numeric(ipos['Shares Offered'], errors='coerce')


In [ ]:
ipos['Withdrawn Value'] = ipos['Shares Offered'] * ipos['Avg. price']

ipos['Withdrawn Value'].isna().value_counts()

,count
Withdrawn Value,
False,71
True,29


In [ ]:
withdrawn_by_class = (
    ipos.groupby('Company Class')['Withdrawn Value']
    .sum()
    .sort_values(ascending=False)
)

print(withdrawn_by_class)


Company Class
Acq.Corp    4.021000e+09
Inc         2.257164e+09
Other       7.679200e+08
Limited     5.497346e+08
Holdings    7.500000e+07
Group       3.378750e+07
Name: Withdrawn Value, dtype: float64


In [ ]:
top_class = withdrawn_by_class.idxmax()
top_value = withdrawn_by_class.max()

print(f"The class with the highest total withdrawn value is: {top_class} (${top_value:,.2f})")


The class with the highest total withdrawn value is: Acq.Corp ($4,021,000,000.00)


### Question 2:   [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)


**What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?**

The goal is to replicate the large-scale `yfinance` OHLCV data download and perform basic financial calculations on IPO stocks.


#### Steps:

1. Using the same approach as in Question 1, download the IPOs in 2024 from:  
   [https://stockanalysis.com/ipos/2024/](https://stockanalysis.com/ipos/2024/)  
   Filter to keep only those IPOs **before 1 June 2024** (first 5 months of 2024).  
   ➤ You should have **75 tickers**.

2.  Use **Code Snippet 7** to download daily stock data for those tickers (via `yfinance`).  
   Make sure you understand how `growth_1d` ... `growth_365d`, and volatility columns are defined.  
   Define a new column `growth_252d` representing growth after **252 trading days** (~1 year), in addition to any other growth periods you already track.


3. Calculate the Sharpe ratio assuming a risk-free rate of **4.5%**:

   ```python
   stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']
   ```

   ⚠️ **IMPORTANT** Please use the original version of annualized volatility calculation (it was later corrected to another formula):
   ```python
   stocks_df['volatility'] =   stocks_df['Close'].rolling(30).std() * np.sqrt(252)
   ```
4. Filter the DataFrame to keep data only for the trading day:  
   **‘2025-06-06’**

   Compute descriptive statistics (e.g., `.describe()`) for these columns:  
   - `growth_252d`  
   - `Sharpe`

   You should observe:  
   - `growth_252d` is defined for **71 out of 75 stocks** (some IPOs are too recent or data starts later).  
   - Median `growth_252d` is approximately **0.75** (indicating a 25% decline), while mean is about **1.15**, showing a bias towards high-growth companies pushing the average up.

5. **Answer:**  
   - What is the **median Sharpe ratio** for these 71 stocks?  
   - Note: Positive `Sharpe` means growth exceeding the risk-free rate of 4.5%.  
   - [Additional] Do you observe the **same top 10 companies** when sorting by `growth_252d` versus sorting by `Sharpe`?

In [ ]:

def get_ipos_by_year(year: int) -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/{year}/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [ ]:

ipos_2024 = get_ipos_by_year(2024)
ipos_2024.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      225 non-null    object
 1   Symbol        225 non-null    object
 2   Company Name  225 non-null    object
 3   IPO Price     225 non-null    object
 4   Current       225 non-null    object
 5   Return        225 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB


In [ ]:
ipos_2024['IPO Date'] = pd.to_datetime(ipos_2024['IPO Date'], errors='coerce')

ipos_2024_filter = ipos_2024[
    (ipos_2024['IPO Date'] < '2024-06-01') &
    (ipos_2024['IPO Price'] != '-')
].copy()

ALL_TICKERS=ipos_2024_filter['Symbol'].values
print('Tickets:',len(ALL_TICKERS))
ALL_TICKERS


Tickets: 75


array(['BOW', 'HDL', 'RFAI', 'JDZG', 'RAY', 'BTOC', 'ZK', 'GPAT', 'PAL',
       'SVCO', 'NNE', 'CCIX', 'VIK', 'ZONE', 'LOAR', 'MRX', 'RBRK', 'NCI',
       'MFI', 'YYGH', 'TRSG', 'CDTG', 'CTRI', 'IBTA', 'MTEN', 'TWG',
       'ULS', 'PACS', 'MNDR', 'CTNM', 'MAMO', 'ZBAO', 'BOLD', 'MMA',
       'UBXG', 'IBAC', 'AUNA', 'BKHA', 'LOBO', 'RDDT', 'ALAB', 'INTJ',
       'RYDE', 'LGCL', 'SMXT', 'VHAI', 'DYCQ', 'CHRO', 'UMAC', 'HLXB',
       'MGX', 'TBBB', 'TELO', 'KYTX', 'PMNT', 'AHR', 'LEGT', 'ANRO',
       'GUTS', 'AS', 'FBLG', 'AVBP', 'BTSG', 'HAO', 'CGON', 'YIBO', 'JL',
       'SUGP', 'JVSA', 'KSPI', 'CCTG', 'PSBD', 'SYNX', 'SDHC', 'ROMA'],
      dtype=object)

In [ ]:

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker, end=', ')

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  historyPrices['Close'] = historyPrices['Close'].ffill()

  # historical returns
  for i in [1,3,7,30,90,252,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)

  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 BOW, 1 HDL, 2 RFAI, 3 JDZG, 4 RAY, 5 BTOC, 6 ZK, 7 GPAT, 8 PAL, 9 SVCO, 10 NNE, 11 CCIX, 12 VIK, 13 ZONE, 14 LOAR, 15 MRX, 16 RBRK, 17 NCI, 18 MFI, 19 YYGH, 20 TRSG, 21 CDTG, 22 CTRI, 23 IBTA, 24 MTEN, 25 TWG, 26 ULS, 27 PACS, 28 MNDR, 29 CTNM, 30 MAMO, 31 ZBAO, 32 BOLD, 33 MMA, 34 UBXG, 35 IBAC, 36 AUNA, 37 BKHA, 38 LOBO, 39 RDDT, 40 ALAB, 41 INTJ, 42 RYDE, 43 LGCL, 44 SMXT, 45 VHAI, 46 DYCQ, 47 CHRO, 48 UMAC, 49 HLXB, 50 MGX, 51 TBBB, 52 TELO, 53 KYTX, 54 PMNT, 55 AHR, 56 LEGT, 57 ANRO, 58 GUTS, 59 AS, 60 FBLG, 61 AVBP, 62 BTSG, 63 HAO, 64 CGON, 65 YIBO, 66 JL, 67 SUGP, 68 JVSA, 69 KSPI, 70 CCTG, 71 PSBD, 72 SYNX, 73 SDHC, 74 ROMA, 

In [ ]:
stocks_df['volatility'] = stocks_df['Close'].rolling(30).std() * np.sqrt(252)

stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']

stocks_df



,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_252d,growth_365d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future,Sharpe
0,23.000000,24.270000,22.139999,23.799999,3335800,0.0,0.0,BOW,2024,5,...,NaN,NaN,1.092017,NaN,NaN,0,0.089496,NaN,1,NaN
1,24.260000,26.150000,23.980000,25.700001,990500,0.0,0.0,BOW,2024,5,...,NaN,NaN,0.998054,NaN,NaN,0,0.084436,NaN,0,NaN
2,25.850000,26.879999,25.075001,26.480000,555100,0.0,0.0,BOW,2024,5,...,NaN,NaN,1.001133,NaN,NaN,0,0.068165,NaN,1,NaN
3,26.440001,26.490000,25.500999,26.290001,302700,0.0,0.0,BOW,2024,5,...,NaN,NaN,0.987828,NaN,NaN,0,0.037619,NaN,0,NaN
4,27.209999,27.209999,25.500000,26.139999,200900,0.0,0.0,BOW,2024,5,...,NaN,NaN,1.037490,NaN,NaN,0,0.065417,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23466,2.850000,2.935000,2.790000,2.795000,10200,0.0,0.0,ROMA,2025,6,...,5.008960,NaN,NaN,3.087,2.89955,1,0.051878,9.469241,0,0.524219
23467,2.883000,2.900000,2.730000,2.790000,33600,0.0,0.0,ROMA,2025,6,...,5.157116,NaN,NaN,2.990,2.93555,1,0.060932,9.196931,0,0.555850
23468,2.960000,3.120000,2.700000,2.780000,123300,0.0,0.0,ROMA,2025,6,...,5.325670,NaN,NaN,2.911,2.97355,0,0.151079,9.001998,0,0.586611
23469,2.760000,2.927000,2.700000,2.850000,170300,0.0,0.0,ROMA,2025,6,...,5.317164,NaN,NaN,2.826,3.01655,0,0.079649,8.804717,0,0.598789


In [ ]:

stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

filter_stocks = stocks_df[stocks_df.Date == pd.to_datetime('2025-06-06')]
filter_stocks[['growth_252d', 'Sharpe']].describe()




,growth_252d,Sharpe
count,71.000000,71.000000
mean,1.152897,0.287253
std,1.406018,0.519513
min,0.024970,-0.079677
25%,0.293422,0.039684
50%,0.758065,0.080707
75%,1.362736,0.311507
max,8.097413,2.835668


In [ ]:
median_sharpe = filter_stocks['Sharpe'].dropna().median()
print(f"Median Sharpe ratio: {median_sharpe:.3f}")


Median Sharpe ratio: 0.081


### Question 3: [IPO] ‘Fixed Months Holding Strategy’

**What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?**  
(*Assume you buy at the close of the first trading day and sell after a fixed number of trading days.*)


---

#### Goal:
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

---

#### Steps:

1. **Start from the existing DataFrame** from Question 2 (75 tickers from IPOs in the first 5 months of 2024).  

   Add **12 new columns**:  
   `future_growth_1m`, `future_growth_2m`, ..., `future_growth_12m`  
   *(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)*  
   This logic is similar to `historyPrices['growth_future_30d']` from **Code Snippet 7**, but extended to longer timeframes.

2. **Determine the first trading day** (`min_date`) for each ticker.  
   This is the earliest date in the data for each stock.

3. **Join the data**:  
   Perform an **inner join** between the `min_date` DataFrame and the future growth data on both `ticker` and `date`.  
   ➤ You should end up with **75 records** (one per IPO) with all 12 `future_growth_...` fields populated.

4. **Compute descriptive statistics** for the resulting DataFrame:  
   Use `.describe()` or similar to analyze each of the 12 columns:  
   - `future_growth_1m`  
   - `future_growth_2m`  
   - ...  
   - `future_growth_12m`  

5. **Determine the best holding period**:  
   - Find the number of months **(1 to 12)** where the **average (mean)** future growth is **maximal**.  
   - This optimal month shows an uplift of **>1%** compared to all others.  
   - Still, the average return remains **less than 1** (i.e., expected return is less than doubling your investment).

In [ ]:
def add_future_growth_columns(df, max_months=12, days_per_month=21):
    df_sorted = df.sort_values(['Ticker', 'Date'])
    df_sorted = df_sorted.set_index('Date')

    for m in range(1, max_months+1):
        df_sorted[f'growth_future_{m}m'] = (
            df_sorted.groupby('Ticker')['Close'].shift(-21 * m) / df_sorted['Close']
        )
    return df_sorted.reset_index()


historyPrices = add_future_growth_columns(stocks_df)
historyPrices

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,...,growth_future_3m,growth_future_4m,growth_future_5m,growth_future_6m,growth_future_7m,growth_future_8m,growth_future_9m,growth_future_10m,growth_future_11m,growth_future_12m
0,2024-02-07,12.085785,12.471402,11.878869,12.433781,12732800,0.0,0.0,AHR,2024,...,1.060660,1.125316,1.214961,1.308118,1.753552,1.963886,1.988395,2.257203,2.259751,2.273312
1,2024-02-08,12.245675,12.452591,12.226865,12.283297,1630300,0.0,0.0,AHR,2024,...,1.094691,1.156243,1.267090,1.324937,1.817033,1.951132,2.105590,2.256846,2.197812,2.230109
2,2024-02-09,12.236271,12.650103,12.179839,12.339728,2062100,0.0,0.0,AHR,2024,...,1.086582,1.157160,1.262084,1.352007,1.803991,1.927869,2.077638,2.219440,2.180527,2.207050
3,2024-02-12,13.120365,13.694088,12.320917,12.367943,1161200,0.0,0.0,AHR,2024,...,1.090294,1.151425,1.278880,1.390634,1.864410,1.953675,2.103896,2.231068,2.194798,2.278196
4,2024-02-13,12.367943,12.753560,12.236270,12.556049,1631700,0.0,0.0,AHR,2024,...,1.067100,1.131888,1.289178,1.399258,1.905472,1.933801,2.028534,2.218782,2.205361,2.222738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23466,2025-06-17,2.990000,3.240000,2.915000,3.160000,32700,0.0,0.0,ZONE,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23467,2025-06-18,3.160000,3.460000,3.078000,3.230000,77300,0.0,0.0,ZONE,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23468,2025-06-20,3.160000,3.525000,3.010000,3.180000,122900,0.0,0.0,ZONE,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23469,2025-06-23,3.210000,3.446000,3.050000,3.210000,21200,0.0,0.0,ZONE,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
min_dates = historyPrices.groupby('Ticker')['Date'].min().reset_index()



In [ ]:
joined_df = pd.merge(min_dates, historyPrices, on=['Ticker', 'Date'], how='inner')

joined_df


,Ticker,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Year,...,growth_future_3m,growth_future_4m,growth_future_5m,growth_future_6m,growth_future_7m,growth_future_8m,growth_future_9m,growth_future_10m,growth_future_11m,growth_future_12m
0,AHR,2024-02-07,12.085785,12.471402,11.878869,12.433781,12732800,0.0,0.0,2024,...,1.060660,1.125316,1.214961,1.308118,1.753552,1.963886,1.988395,2.257203,2.259751,2.273312
1,ALAB,2024-03-20,52.560001,63.500000,50.610001,62.029999,16843300,0.0,0.0,2024,...,1.018217,0.853780,0.631307,0.759794,1.078027,1.457037,1.974367,1.982105,1.381912,1.189424
2,ANRO,2024-02-02,22.000000,23.270000,20.000000,20.700001,2386300,0.0,0.0,2024,...,0.686957,0.546377,0.563768,0.438164,0.647343,0.492270,0.192271,0.203865,0.222705,0.202899
3,AS,2024-02-01,13.400000,13.800000,13.100000,13.400000,18656400,0.0,0.0,2024,...,1.080597,1.143284,0.935075,0.802239,0.999254,1.236567,1.332090,1.950000,2.152985,2.362687
4,AUNA,2024-03-22,9.510000,10.320000,9.300000,9.600000,9046900,0.0,0.0,2024,...,0.801042,0.919792,0.782292,0.767708,0.723958,0.733333,0.706250,0.857292,0.850000,0.752083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,YIBO,2024-01-25,4.000000,4.000000,2.400000,2.790000,875500,0.0,0.0,2024,...,0.734767,0.788530,0.723656,1.044803,0.777778,0.957706,0.996416,1.290323,1.326165,1.842294
71,YYGH,2024-04-22,3.880000,4.700000,2.580000,2.900000,990300,0.0,0.0,2024,...,0.246897,0.255172,0.315517,0.431034,0.631034,0.675862,0.634483,0.606897,0.617241,0.355172
72,ZBAO,2024-04-02,4.000000,4.000000,3.000000,3.700000,755000,0.0,0.0,2024,...,1.075676,1.037838,0.891892,1.005405,0.851351,0.770270,0.424324,0.418919,0.370270,0.300541
73,ZK,2024-05-10,26.000000,29.360001,25.510000,28.260000,9709800,0.0,0.0,2024,...,0.508139,0.598726,0.869073,0.878627,0.998231,0.961076,0.877212,1.105096,0.756546,1.018401


In [ ]:
future_growth_cols = [f'future_growth_{i}m' for i in range(1, 13)]
growth_stats = joined_df[future_growth_cols].describe()
print(growth_stats)


       future_growth_1m  future_growth_2m  future_growth_3m  future_growth_4m  \
count         75.000000         75.000000         75.000000         75.000000   
mean           0.927259          0.940544          0.833825          0.825087   
std            0.346261          0.574546          0.409762          0.401967   
min            0.098947          0.073800          0.060947          0.045368   
25%            0.778984          0.685815          0.511212          0.517233   
50%            0.977000          1.000000          0.927500          0.909091   
75%            1.046509          1.154013          1.069085          1.134300   
max            2.646505          4.874759          2.040000          1.605000   

       future_growth_5m  future_growth_6m  future_growth_7m  future_growth_8m  \
count         75.000000         75.000000         75.000000         75.000000   
mean           0.803768          0.864186          0.847149          0.832982   
std            0.488348    

In [ ]:
monthly_means = joined_df[joined_df.columns[-12:]].describe().iloc[1].sort_values(ascending=False)
best_month = monthly_means.index[0]
best_value = monthly_means.iloc[0]

print(f"Best month: {best_month}")
print(f"Mean return: {best_value:.4f}")


Best month: growth_future_2m
Mean return: 0.9405


### Question 4: [Strategy] Simple RSI-Based Trading Strategy


**What is the total profit (in $thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?**


---

#### Goal:
Apply a simple rule-based trading strategy using the **Relative Strength Index (RSI)** technical indicator to identify oversold signals and calculate profits.

---


#### Steps:

1. **Run the full notebook from Lecture 2 (33 stocks)**  
   - Ensure you can generate the merged DataFrame containing:  
     - OHLCV data  
     - Technical indicators  
     - Macro indicators  
   - Focus on getting **RSI** computed using **Code Snippets 8 and 9**.  
   - This process is essential and will help during the capstone project.

2. ⚠️ **IMPORTANT** Please use this file to solve the Home Assignment (**all next steps**)

   Download precomputed data using this snippet:

   ```python
   import gdown
   import pandas as pd

   file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
   gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
   df = pd.read_parquet("data.parquet", engine="pyarrow")

3. **RSI Strategy Setup:**  
   - RSI is already available in the dataset as a field.  
   - The threshold for **oversold** is defined as `RSI < 25`.

4. **Filter the dataset by RSI and date:**  
   ```python
   rsi_threshold = 25
   selected_df = df[
       (df['rsi'] < rsi_threshold) &
       (df['Date'] >= '2000-01-01') &
       (df['Date'] <= '2025-06-01')
   ]
5. **Calculate Net Profit Over 25 Years:**  
   - Total number of trades: **1568**  
   - For each trade, you invest **$1000**  
   - Use the 30-day forward return (`growth_future_30d`) to compute net earnings:  
     ```python
     net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
     ```

   - **Final Answer:**  
     What is the **net income in $K** (i.e., in thousands of dollars) that could be earned using this RSI-based oversold strategy from 2000–2025?

In [ ]:
import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")


Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=bdd8c2ec-bc48-4f48-8cbb-0f9cffcf0f8a
To: /content/data.parquet
100%|██████████| 130M/130M [00:01<00:00, 77.5MB/s]


In [ ]:
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]


In [ ]:
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()

net_income_k = net_income / 1000

print(f"Net income over 25 years: ${net_income_k:.2f}K")


Net income over 25 years: $24.30K


### Q5. [Exploratory, Optional] Predicting a Positive-Return IPO

Most of the strategies for investing in IPOs deliver **negative average and median returns** (and even 75% quantiles).

**Question:**  
How would you change the strategy if you want to **increase the profitability**?

> This is an open-ended brainstorming question — propose ideas for identifying IPOs with positive future returns or building a more effective trading strategy.

**To improve profitability when investing in IPOs, delay entry until post-IPO stabilization, apply quality and technical filters (like RSI or momentum), avoid bear markets, and optionally use machine learning to predict positive-return candidates.**

